In [ ]:
import numpy as np

# AIS implementation from https://github.com/wmingwei/restricted-boltzmann-machine-deep-belief-network-deep-boltzmann-machine-in-pytorch/blob/fd21ac2461564252de01f99425ddabd5eec11f41/RBM/ais.py#L39

In [ ]:
def ais(rbm, step = 100, M = 100, seed = None):

    W = rbm.W.data.numpy().T
    v_bias = rbm.v_bias.data.numpy()
    h_bias = rbm.h_bias.data.numpy()
    
    logZ0 = np.log((1+np.exp(v_bias))).sum() + np.log(1+np.exp(h_bias)).sum()
    ratio = []
    for i in range(M):
        ratio.append(mcmc(step, seed = seed,  W = W, h_bias = h_bias, v_bias = v_bias))

    ratio = np.array(ratio).reshape(len(ratio),1)
    logZ = logZ0 + logmeanexp(ratio, axis = 0)

    return logZ

def mcmc(step, seed, W, h_bias, v_bias):

    np.random.seed(seed)

    v = np.random.binomial(1, p=1/(1+np.exp(-v_bias))).reshape(1,-1)

    logw = 0
    for k in range(step):
        logp_k = -free_energy(v, k*1.0/step*W, h_bias, v_bias)
        logp_k1 = -free_energy(v, (k+1)*1.0/step*W, h_bias, v_bias)
        logw += logp_k1 - logp_k

        
        p_h, h = v_to_h(v, (k+1)*1.0/step*W, h_bias)
        p_v, v = h_to_v(h, (k+1)*1.0/step*W, v_bias)

    return logw

def free_energy(v, W, h_bias, v_bias):

    Wv = np.clip(np.matmul(v,W) + h_bias,-80,80)
    hidden = np.log(1+np.exp(Wv)).sum(1)
    vbias = np.matmul(v, v_bias.T).reshape(hidden.shape)
    return -hidden-vbias


def logmeanexp(x, axis=None):
    
    x = np.asmatrix(x)
    if not axis:
        n = len(x)
    else:
        n = x.shape[axis]
    
    x_max = x.max(axis)
    return (x_max + np.log(np.exp(x-x_max).sum(axis)) - np.log(n)).A

In [ ]:
import tensorflow as tf
import numpy as np

n_vis = 10
n_hidd = 20
n_class = 30
batch_size = 100

input_data = np.random.rand(batch_size,n_vis)
W = np.random.rand(n_vis,n_hidd)
hidden_bias = np.random.rand(n_hidd)
U = np.random.rand(n_class,n_hidd)

tf.nn.sigmoid(tf.reshape((tf.repeat(tf.einsum("ni,ij->nj",input_data,W)+hidden_bias,n_class)),shape = (batch_size,n_class,n_hidd))+U)

In [20]:
positive_sum = tf.zeros([batch_size,self.n_hidden])
class_weight_grad = tf.zeros([self.n_class,self.n_hidden])

for i,c in enumerate(class_label):
    positive_sum[i] += o_y_j[i, : , c]
    class_weight_grad[c ,:] += positive_sum[i]

<tf.Tensor: shape=(20,), dtype=float64, numpy=
array([2.73073274, 2.9912447 , 3.71027033, 3.84297624, 2.84562585,
       2.21961976, 3.12417395, 2.44894882, 2.60629273, 3.94587099,
       3.91821798, 3.26617206, 2.06384555, 2.11889287, 2.81002009,
       3.08849918, 2.47702441, 3.32047782, 3.2167166 , 1.92684327])>

# Correct LL(positive)?

In [23]:
from Save_Load import from_dataset_to_array

print("This may take some time")
CORRUPTED,CORRUPTED_LABEL = from_dataset_to_array(data_name = "shot_noise") 
DATA,DATA_LABEL = from_dataset_to_array(data_name = "identity")

val_split = 1/3
train_data,val_data = train_test_split(DATA, test_size = val_split,random_state = 42)
train_corrupted,val_corrupted = train_test_split(CORRUPTED, test_size = val_split,random_state = 42)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 1, 2])>

In [ ]:
EPOCHS = 11 # Time to train for each epoch approx 1 minute, and each model is repeated 10 times, thus, elapsed time will be [0.15,0.22]*epochs hours.
             # For 100 epochs, maximum time limit should be 24 hours. 
             # Partitions(Walltime in hours)-> short(0.5), medium(6), large(120)

lr = 0.01
momentum = 0.5
batch_size = 2**6
SEED = 9

n_hidden = 1000#81*5

window = 1
stride = 2

windowList = []#[2,3,4]
strideList = []#[4,4,4]


dim = 28

kwargs = vars_to_dict(batch_size = batch_size,n_visible = 28*28,n_hidden = n_hidden,
                      window = window, stride = stride, windowList = windowList, strideList = strideList,
                      epochs = EPOCHS,k_gibbs = 1,
                      MAIN = MAIN,metrics = metrics,train_data=train_data,val_data=val_data,
                      train_corrupted=train_corrupted,val_corrupted=val_corrupted,seed = SEED,
                      lr = lr,momentum = momentum, dtype = dtype)

In [ ]:
from Save_Load import save_rbm,load_rbm
n_hid = 100 #1000,400,100
rbm = RBM(**kwargs)
aux = load_rbm("RBM_n_hidden_%s_method_LL_seed_9_epochs_100_lr_01_momentum_05_batch_size_64"%n_hid,MAIN=MAIN)
rbm.M = 2**9
rbm.W = aux.W
rbm.visible_bias = aux.visible_bias
rbm.hidden_bias = aux.hidden_bias


rbm.approx_log_Z = rbm.AIS_log_Z()

In [ ]:
rbm.approx_log_Z
rbm.approximated_LL(train_data)

# Neighbourhoods from covariance matrix

In [ ]:
from Save_Load import from_dataset_to_array
import matplotlib.pyplot as plt

print("This may take some time")
CORRUPTED,CORRUPTED_LABEL = from_dataset_to_array(data_name = "shot_noise") 
DATA,DATA_LABEL = from_dataset_to_array(data_name = "identity")

val_split = 1/3
train_data,val_data = train_test_split(DATA, test_size = val_split,random_state = 42)
train_corrupted,val_corrupted = train_test_split(CORRUPTED, test_size = val_split,random_state = 42)

In [ ]:
from sklearn.covariance import empirical_covariance
cov = empirical_covariance(DATA[:10000])

In [ ]:
m = 60000
cov = np.cov(DATA[:m].T)

In [ ]:
import seaborn as sn

plt.figure(figsize = (10,7))
fig = sn.heatmap(cov)

In [ ]:
def return_p_biggest(a,p):
    #n = int(p*len(a))
    #return a.argsort()[-n:][::-1]
    m = np.max(a)
    #print(a[a.argsort()[-3:][::-1]])
    for i,ind in enumerate(a.argsort()[::-1]):
        if a[ind]/m<p:
            break
    return a.argsort()[-i:][::-1]
p = 0.25
eps = 0.02
mat = np.abs(cov)
mat[np.where(mat<eps)] = 0

In [ ]:
for i in range(len(mat)):
    if np.any(mat[i,:]):
        mat[i,return_p_biggest(mat[i,:],p)]=1
        mat[i,i]=1


In [ ]:
k = 0
for i in range(len(mat)):
    if np.any(mat[i,:]):
        k+=1
print(k)

In [ ]:
plt.figure(figsize = (10,7))
fig = sn.heatmap(mat)

In [ ]:
ind = int(784/2)+5
print(mat[ind,:])
plt.imshow(mat[ind,:].reshape((28,28)))

In [ ]:
eps = 0.025
plt.figure(figsize = (10,7))
fig = sn.heatmap(np.abs(cov)>eps)

In [ ]:
n = 28
dim = 28
figsize = kwargs.get("figsize",(20,20))
fig, axs = plt.subplots(n, n, figsize=figsize)

eps = 0.001
m = np.abs(cov)>eps #m = cov

for i in range(n):
    for j in range(n):
        axs[i,j].imshow(m[n*i+j].reshape((dim,dim)),cmap = plt.get_cmap('gray'))
        axs[i,j].axis('off')
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
plt.imshow(np.abs(cov[28*14+14,:].reshape((28,28))),cmap="gray")